In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sns
from matplotlib import pyplot as plt
import pyGadget

In [ ]:
sns.set_context('poster')

In [ ]:
rad = 1e5
views = {'native':None, 'face (xy)':'face', 'xz':'xz', 'yz':'yz'}
particles = {'x':[], 'y':[], 'z':[], 'n':[], 'view':[], 'snapshot':[]}
for sim, snap in [('vanilla',355), ('xr_tau_J0',306), ('xr_tau_J1',327), ('xr_tau_J2',271), ('xr_tau_J3',235)]:
    sim = pyGadget.sim.Simulation('stampede/'+sim,length='AU')
    s = sim.load_snapshot(snap)
    s.gas.load_coords()
    for v in ['native', 'face (xy)', 'xz', 'yz']:
        vv = views[v]
        s.gas.orient_box(centering='max', view=vv)
        r2d = np.linalg.norm(s.gas.coordinates[:,:2], axis=1)
        pos, n = pyGadget.analyze.data_slice(r2d < rad, s.gas.coordinates, s.gas.ndensity)
        particles['x'].extend(pos[:,0])
        particles['y'].extend(pos[:,1])
        particles['z'].extend(pos[:,2])
        particles['n'].extend(n)
        particles['view'].extend([v for i in range(n.size)])
        particles['snapshot'].extend([snap for i in range(n.size)])

    s.gas.cleanup()
    s.close()

In [ ]:
particles = pd.DataFrame(particles)
particles['log z'] = np.log10(particles['z'])
particles['log n'] = np.log10(particles['n'])

In [ ]:
g = sns.FacetGrid(particles, col='view', hue='snapshot', col_wrap=2)
g.map(plt.scatter, 'log z', 'log n', s=5, alpha=.5)#, linewidth=.5, edgecolor="white")

In [ ]:
g.fig.savefig('figures/optical_depth/views-overlay.png', bbox_inches='tight')

# Interpolating Column with griddata

In [ ]:
xmax = ymax = 1
xmin = ymin = -1
zmax = 1e7
zmin = 0
(zmax-zmin)

In [ ]:
grid_x, grid_y, grid_z = np.mgrid[xmin:xmax:3j, ymin:ymax:3j, zmin:zmax:10000001j]

In [ ]:
grid_z.shape, grid_z.size

In [ ]:
z = grid_z[1,1]
z[1] - z[0] #Double-check grid spacing!

In [ ]:
sigma = 1.14e-23
optical_depth = {'z':[], 'n':[], 'N':[], 'tau':[], 'view':[], 'snapshot':[]}
for snap in [355, 306, 327, 271, 235]:
    for view in ['native', 'face (xy)', 'xz', 'yz']:
        selection = particles[(particles['view'] == view) & (particles['snapshot'] == snap)]
        pos = np.column_stack([selection['x'].values, selection['y'].values, selection['z'].values])
        n = selection['n'].values
        grid_n = griddata(pos, n, (grid_x, grid_y, grid_z), method='linear')
        nz = grid_n[1,1]
        N = nz[::-1].cumsum() * 1.49597871e13 #convert from AU to cm
        tau = N[::-1] * sigma
        print snap, view, tau[0]
        optical_depth['z'].extend(z)
        optical_depth['n'].extend(nz)
        optical_depth['N'].extend(N[::-1])
        optical_depth['tau'].extend(tau)
        optical_depth['view'].extend([view for i in range(z.size)])
        optical_depth['snapshot'].extend([snap for i in range(z.size)])
optical_depth = pd.DataFrame(optical_depth)

In [ ]:
store = pd.HDFStore("/home/r900-4/jhummel/data/simdata/xrays/optical_depth_referee.hdf5")

In [ ]:
store['particles'] = particles
store['optical_depth'] = optical_depth

In [ ]:
store.close()

In [ ]:
optical_depth[optical_depth['z'] == 0]

In [ ]:
g = sns.FacetGrid(optical_depth, col='snapshot', hue='view', col_wrap=2)
g.map(plt.loglog, 'z', 'tau')
g.set(xlim=(1,5e6), ylim=(2e-4, 5e4))
g.set_xlabels('z [AU]')
g.set_ylabels('Optical Depth')
g.fig.subplots_adjust(wspace=.02)

In [ ]:
g = sns.FacetGrid(optical_depth, col='view', hue='snapshot', col_wrap=2)
g.map(plt.loglog, 'z', 'tau')
g.set(xlim=(1,5e6), ylim=(2e-4, 5e4))
g.set_xlabels('z [AU]')
g.set_ylabels('Optical Depth')
g.fig.subplots_adjust(wspace=.02)

In [ ]:
g.fig.savefig('figures/optical_depth/optical_depth-r.png', bbox_inches='tight')

In [ ]:
g = sns.FacetGrid(optical_depth, col='snapshot', hue='view', col_wrap=2)
g.map(plt.loglog, 'n', 'tau')
g.set(xlim=(1e12,.1), ylim=(2e-4, 5e4))
g.set_xlabels('n [cm$^{-3}$]')
g.set_ylabels('Optical Depth')
g.fig.subplots_adjust(wspace=.02)

In [ ]:
g = sns.FacetGrid(optical_depth, col='view', hue='snapshot', col_wrap=2)
g.map(plt.loglog, 'n', 'tau')
g.set(xlim=(1e12,.1), ylim=(2e-4, 5e4))
g.set_xlabels('n [cm$^{-3}$]')
g.set_ylabels('Optical Depth')
g.fig.subplots_adjust(wspace=.02)

In [ ]:
g.fig.savefig('figures/optical_depth/optical_depth-n.png', bbox_inches='tight')